# 【AAPlot 多图版 - 优化版】
## 用于对Spike2输出的txt文件绘制时间序列图

### 主要改进：
1. **配置管理** - 所有参数集中管理，便于修改
2. **错误处理** - 增加文件检查和异常处理
3. **颜色配置** - 支持多种颜色方案，便于切换
4. **路径管理** - 支持多种路径设置方式

### 数据格式要求：
1. 输入数据为Spike2直接输出的Spreadsheet类文件，格式为.csv或.txt
2. 文件内容应为三列：
   - 第一列为时间
   - 第二列为数据点
   - 第三列为药物注射时间打标，数据中该行显示为1

### 环境要求：
- Python3.12.7
- pandas, numpy, matplotlib, seaborn, scipy
- ipykernel


## Step1: 配置参数设置


In [ ]:
# ========== 配置参数区域 ==========
# 在这里修改您的参数设置

# 1. 数据文件夹路径设置
FOLDER_PATH = r'D:\Science\Experiment data\eCB sensors\Selective sensor\FiberPhotometry\Drug intake\NAcSh_All_data_in_1_folder\01_Summary\2-AG\cocaine'

# 2. 颜色方案设置 (选择其中一个方案)
COLOR_SCHEME = {
    # 方案1: 经典配色
    'classic': {
        'individual': 'lightgrey',
        'mean': 'darkgoldenrod',
        'error_band': 'darkgoldenrod'
    },
    # 方案2: 蓝色系
    'blue': {
        'individual': 'lightblue',
        'mean': 'steelblue',
        'error_band': 'steelblue'
    },
    # 方案3: 红色系
    'red': {
        'individual': 'lightcoral',
        'mean': 'darkred',
        'error_band': 'darkred'
    },
    # 方案4: 绿色系
    'green': {
        'individual': 'lightgreen',
        'mean': 'darkgreen',
        'error_band': 'darkgreen'
    },
    # 方案5: 紫色系
    'purple': {
        'individual': 'plum',
        'mean': 'purple',
        'error_band': 'purple'
    }
}

# 当前使用的颜色方案 (修改这里切换颜色)
CURRENT_COLOR_SCHEME = 'classic'  # 可选: 'classic', 'blue', 'red', 'green', 'purple'

# 3. 图表参数设置
PLOT_CONFIG = {
    'figsize': (8, 3),
    'linewidth_individual': 1,
    'linewidth_mean': 3,
    'linewidth_vertical': 3,
    'alpha_error_band': 0.2,
    'font_family': 'Arial',
    'xlabel_size': 20,
    'ylabel_size': 20,
    'tick_size': 18
}

# 4. 坐标轴设置
AXIS_CONFIG = {
    'x_limits': (-0.25, 1.5),
    'y_limits': (-5, 15),
    'x_ticks': np.arange(-0.5, 2.1, 0.5),
    'y_ticks': np.arange(-5, 15, 5)
}

# 5. 分析参数设置
ANALYSIS_CONFIG = {
    'smoothing_window': 2001,
    'smoothing_polyorder': 2,
    'start_time': 300,   # 秒
    'end_time': 2100     # 秒
}

# 6. 输出设置
OUTPUT_CONFIG = {
    'save_plot': True,
    'save_metrics': True,
    'save_aligned_data': False,
    'plot_format': 'svg',
    'plot_dpi': 300
}

# ========== 配置验证 ==========
def validate_config():
    """验证配置参数"""
    if not os.path.exists(FOLDER_PATH):
        print(f"⚠️ 警告: 文件夹不存在 - {FOLDER_PATH}")
        return False
    
    if CURRENT_COLOR_SCHEME not in COLOR_SCHEME:
        print(f"⚠️ 警告: 未知的颜色方案 - {CURRENT_COLOR_SCHEME}")
        return False
    
    print(f"✅ 配置验证通过")
    print(f"📁 数据文件夹: {FOLDER_PATH}")
    print(f"🎨 颜色方案: {CURRENT_COLOR_SCHEME}")
    return True

# 验证配置
validate_config()


## Step2: 导入必要库


In [ ]:
# 导入必要库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import warnings
from scipy.signal import savgol_filter

# 忽略警告信息
warnings.filterwarnings('ignore')

print("✅ 所有库导入成功")


## Step3: 数据加载和处理


In [ ]:
# ========== 数据加载函数 ==========
def load_and_process_data(folder_path):
    """加载和处理数据文件"""
    try:
        # 检查文件夹是否存在
        if not os.path.exists(folder_path):
            raise FileNotFoundError(f"文件夹不存在: {folder_path}")
        
        # 获取所有txt文件
        txt_files = glob.glob(os.path.join(folder_path, '*.txt'))
        txt_files.sort()
        
        if not txt_files:
            raise FileNotFoundError(f"在 {folder_path} 中未找到.txt文件")
        
        print(f"📁 找到 {len(txt_files)} 个数据文件")
        
        # 读取所有文件
        all_data = []
        for i, file in enumerate(txt_files):
            try:
                df = pd.read_csv(file, sep=',')
                if df.shape[1] < 3:
                    print(f"⚠️ 警告: 文件 {os.path.basename(file)} 列数不足")
                    continue
                all_data.append(df)
                print(f"✅ 加载: {os.path.basename(file)} ({len(df)} 行)")
            except Exception as e:
                print(f"❌ 错误: 无法读取 {os.path.basename(file)} - {e}")
        
        if not all_data:
            raise ValueError("没有成功加载任何数据文件")
        
        return all_data
        
    except Exception as e:
        print(f"❌ 数据加载失败: {e}")
        return None

# ========== 数据对齐函数 ==========
def align_data_to_events(all_data):
    """将数据对齐到事件标记"""
    try:
        print("🔄 开始数据对齐...")
        
        # 重算每个dataframe中的时间，以事件标记为0点
        for i, df in enumerate(all_data):
            event_mask = df.iloc[:, 2] == 1
            if not event_mask.any():
                print(f"⚠️ 警告: 文件 {i} 中未找到事件标记")
                continue
            event_time = df.iloc[:, 0][event_mask].iloc[0]
            df.iloc[:, 0] = df.iloc[:, 0] - event_time
        
        # 计算时间间隔
        dt = all_data[0].iloc[1, 0] - all_data[0].iloc[0, 0]
        print(f"⏱️ 时间间隔: {dt:.4f} 秒")
        
        # 找到每个文件中事件标记的位置
        time_0_indices = []
        valid_data = []
        
        for i, df in enumerate(all_data):
            event_mask = df.iloc[:, 2] == 1
            if event_mask.any():
                time_0_index = df.iloc[:, 0][event_mask].index[0]
                time_0_indices.append(time_0_index)
                valid_data.append(df)
        
        if not time_0_indices:
            raise ValueError("没有找到有效的事件标记")
        
        # 计算对齐参数
        max_pre_len = max(time_0_indices)
        max_post_len = max([len(df) - idx for df, idx in zip(valid_data, time_0_indices)])
        
        print(f"📊 对齐参数: 前段最大长度={max_pre_len}, 后段最大长度={max_post_len}")
        
        # 对齐数据
        all_data_aligned = pd.DataFrame()
        
        for i, (df, time_0_idx) in enumerate(zip(valid_data, time_0_indices)):
            signal = df.iloc[:, 1].values
            
            # 对齐前段数据
            pre_signal = signal[:time_0_idx]
            padded_pre = np.full(max_pre_len, np.nan)
            padded_pre[-len(pre_signal):] = pre_signal
            
            # 对齐后段数据
            post_signal = signal[time_0_idx:]
            padded_post = np.full(max_post_len, np.nan)
            padded_post[:len(post_signal)] = post_signal
            
            # 合并信号
            aligned_signal = np.concatenate([padded_pre, padded_post])
            all_data_aligned[f'signal_{i}'] = aligned_signal
        
        # 创建时间轴
        total_length = max_pre_len + max_post_len
        pre_time = np.linspace(-max_pre_len * dt/3600, 0, max_pre_len, endpoint=False)
        post_time = np.linspace(0, max_post_len * dt/3600, max_post_len, endpoint=False)[1:]
        time_axis = np.concatenate([pre_time, [0], post_time])
        
        # 创建最终DataFrame
        final_df = pd.DataFrame({'Time(h)': time_axis})
        
        for i, col in enumerate(all_data_aligned.columns):
            final_df[col] = pd.to_numeric(all_data_aligned[col], errors='coerce')
        
        # 计算统计量
        signal_columns = [col for col in final_df.columns if 'signal_' in col]
        if signal_columns:
            final_df['Mean'] = final_df[signal_columns].mean(axis=1, skipna=True)
            final_df['Std'] = final_df[signal_columns].std(axis=1, skipna=True)
            n_valid = final_df[signal_columns].count(axis=1)
            final_df['StdErr'] = final_df['Std'] / np.sqrt(n_valid)
        
        print(f"✅ 数据对齐完成: {len(signal_columns)} 个信号, {len(final_df)} 个时间点")
        return final_df, dt
        
    except Exception as e:
        print(f"❌ 数据对齐失败: {e}")
        return None, None

# ========== 执行数据处理 ==========
print("🚀 开始数据处理...")
all_data = load_and_process_data(FOLDER_PATH)

if all_data is not None:
    final_df, dt = align_data_to_events(all_data)
    if final_df is not None:
        print("✅ 数据处理完成")
        print(f"📊 数据摘要: {final_df.shape[0]} 行, {final_df.shape[1]} 列")
    else:
        print("❌ 数据处理失败")
else:
    print("❌ 数据加载失败")


## Step4: 数据可视化


In [ ]:
# ========== 可视化函数 ==========
def create_plot(final_df, color_scheme, plot_config, axis_config, folder_path):
    """创建时间序列图"""
    try:
        print("📊 开始创建图表...")
        
        # 获取当前颜色方案
        colors = color_scheme
        
        # 创建图表
        plt.figure(figsize=plot_config['figsize'])
        
        # 获取信号列
        signal_columns = [col for col in final_df.columns if 'signal_' in col]
        
        # 绘制个体信号
        for col in signal_columns:
            plt.plot(final_df['Time(h)'], final_df[col], 
                    color=colors['individual'],
                    linewidth=plot_config['linewidth_individual'],
                    alpha=0.7)
        
        # 绘制均值曲线
        if 'Mean' in final_df.columns:
            plt.plot(final_df['Time(h)'], final_df['Mean'], 
                    color=colors['mean'],
                    linewidth=plot_config['linewidth_mean'],
                    label='Mean')
            
            # 绘制误差带（可选）
            if 'StdErr' in final_df.columns:
                plt.fill_between(final_df['Time(h)'],
                               final_df['Mean'] - final_df['StdErr'],
                               final_df['Mean'] + final_df['StdErr'],
                               color=colors['error_band'],
                               alpha=plot_config['alpha_error_band'])
        
        # 设置字体
        plt.rcParams['font.sans-serif'] = [plot_config['font_family']]
        
        # 设置标签
        plt.xlabel('Time (h)', fontsize=plot_config['xlabel_size'])
        plt.ylabel('z-score', fontsize=plot_config['ylabel_size'])
        
        # 设置刻度
        plt.xticks(axis_config['x_ticks'], fontsize=plot_config['tick_size'])
        plt.yticks(axis_config['y_ticks'], fontsize=plot_config['tick_size'])
        
        # 设置轴范围
        plt.xlim(axis_config['x_limits'])
        plt.ylim(axis_config['y_limits'])
        
        # 添加事件线
        plt.axvline(0, color='black', linestyle='--', 
                   linewidth=plot_config['linewidth_vertical'])
        
        # 设置透明背景
        plt.gcf().patch.set_alpha(0.0)
        plt.gca().patch.set_alpha(0.0)
        sns.despine()
        
        # 保存图表
        if OUTPUT_CONFIG['save_plot']:
            output_fig = os.path.join(folder_path, f'plot_{CURRENT_COLOR_SCHEME}.{OUTPUT_CONFIG["plot_format"]}')
            plt.savefig(output_fig, format=OUTPUT_CONFIG['plot_format'], 
                       bbox_inches='tight', dpi=OUTPUT_CONFIG['plot_dpi'])
            print(f"✅ 图表已保存: {output_fig}")
        
        plt.show()
        print("✅ 图表创建完成")
        
    except Exception as e:
        print(f"❌ 图表创建失败: {e}")

# ========== 执行可视化 ==========
if 'final_df' in locals() and final_df is not None:
    # 获取当前颜色方案
    current_colors = COLOR_SCHEME[CURRENT_COLOR_SCHEME]
    
    # 创建图表
    create_plot(final_df, current_colors, PLOT_CONFIG, AXIS_CONFIG, FOLDER_PATH)
else:
    print("❌ 没有可用的数据用于绘图")


## Step5: 数据分析


In [ ]:
# ========== 数据平滑函数 ==========
def smooth_data(df, window_length=2001, polyorder=2):
    """对数据进行平滑处理"""
    try:
        smoothed_df = df.copy()
        signal_columns = [col for col in df.columns if 'signal_' in col]
        
        for col in signal_columns:
            # 移除NaN值进行平滑
            valid_mask = ~pd.isna(smoothed_df[col])
            if valid_mask.sum() > window_length:
                smoothed_values = savgol_filter(
                    smoothed_df.loc[valid_mask, col].values,
                    window_length, polyorder, mode='nearest'
                )
                smoothed_df.loc[valid_mask, col] = smoothed_values
        
        print(f"✅ 数据平滑完成 (窗口={window_length}, 阶数={polyorder})")
        return smoothed_df
        
    except Exception as e:
        print(f"❌ 数据平滑失败: {e}")
        return df

# ========== 指标计算函数 ==========
def calculate_metrics(df, start_time=300, end_time=2100):
    """计算统计指标"""
    try:
        signal_columns = [col for col in df.columns if 'signal_' in col]
        metrics = {}
        
        # 创建时间掩码
        time_mask = ((df['Time(h)'] >= start_time / 3600) & 
                    (df['Time(h)'] <= end_time / 3600))
        
        for col in signal_columns:
            filtered_data = df.loc[time_mask, col].dropna()
            
            if len(filtered_data) > 0:
                # 计算最大值、最小值
                max_val = filtered_data.max()
                min_val = filtered_data.min()
                
                # 计算曲线下面积 (AUC)
                time_points = df.loc[time_mask, 'Time(h)'].values
                valid_time_mask = ~pd.isna(df.loc[time_mask, col])
                
                if valid_time_mask.sum() > 1:
                    valid_time = time_points[valid_time_mask]
                    valid_data = filtered_data.values
                    auc = np.trapz(valid_data, valid_time)
                else:
                    auc = np.nan
                
                metrics[col] = {
                    'max': max_val,
                    'min': min_val,
                    'area': auc
                }
            else:
                metrics[col] = {'max': np.nan, 'min': np.nan, 'area': np.nan}
        
        metrics_df = pd.DataFrame(metrics).T
        print(f"✅ 指标计算完成: {len(metrics_df)} 个信号")
        return metrics_df
        
    except Exception as e:
        print(f"❌ 指标计算失败: {e}")
        return pd.DataFrame()

# ========== 执行数据分析 ==========
if 'final_df' in locals() and final_df is not None:
    print("🔬 开始数据分析...")
    
    # 数据平滑
    smoothed_df = smooth_data(final_df, 
                              ANALYSIS_CONFIG['smoothing_window'], 
                              ANALYSIS_CONFIG['smoothing_polyorder'])
    
    # 计算指标
    metrics_df = calculate_metrics(smoothed_df, 
                                   ANALYSIS_CONFIG['start_time'], 
                                   ANALYSIS_CONFIG['end_time'])
    
    # 保存指标
    if OUTPUT_CONFIG['save_metrics'] and not metrics_df.empty:
        output_metrics = os.path.join(FOLDER_PATH, f'metrics_{CURRENT_COLOR_SCHEME}.csv')
        metrics_df.to_csv(output_metrics, index=True)
        print(f"✅ 指标已保存: {output_metrics}")
    
    # 保存对齐数据（可选）
    if OUTPUT_CONFIG['save_aligned_data']:
        output_data = os.path.join(FOLDER_PATH, f'aligned_data_{CURRENT_COLOR_SCHEME}.csv')
        final_df.to_csv(output_data, index=False)
        print(f"✅ 对齐数据已保存: {output_data}")
    
    # 显示结果
    print("\n📊 统计指标:")
    print(metrics_df)
    
    # 显示数据摘要
    print(f"\n📋 数据摘要:")
    print(f"   信号数量: {len([col for col in final_df.columns if 'signal_' in col])}")
    print(f"   时间范围: {final_df['Time(h)'].min():.3f} - {final_df['Time(h)'].max():.3f} 小时")
    print(f"   数据点数: {len(final_df)}")
    
else:
    print("❌ 没有可用的数据用于分析")


## 使用说明

### 🎨 如何切换颜色方案：
1. 在 **Step1** 中找到 `CURRENT_COLOR_SCHEME` 变量
2. 将其修改为以下选项之一：
   - `'classic'` - 经典配色（灰色+金色）
   - `'blue'` - 蓝色系
   - `'red'` - 红色系
   - `'green'` - 绿色系
   - `'purple'` - 紫色系
3. 重新运行 **Step4** (数据可视化)

### 📁 如何更换数据文件夹：
1. 在 **Step1** 中找到 `FOLDER_PATH` 变量
2. 修改为您的实际数据文件夹路径
3. 重新运行 **Step3** (数据加载和处理)

### ⚙️ 其他参数调整：
- **图表参数**: 修改 `PLOT_CONFIG` 中的设置
- **坐标轴**: 修改 `AXIS_CONFIG` 中的范围
- **分析参数**: 修改 `ANALYSIS_CONFIG` 中的时间窗口
- **输出设置**: 修改 `OUTPUT_CONFIG` 中的保存选项
